In [ ]:
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

In [ ]:
viral_reference_fastas_dir = mkpath(joinpath(data_dir, "viral-reference-fastas"))

In [ ]:
# run(`ln -s $(homedir())/workspace/JGI/IMG_VR_2022-12-19_7.1/IMGVR_all_nucleotides-high_confidence.fna.gz $(viral_reference_fastas_dir)`)

In [ ]:
# run(`ln -s $(homedir())/workspace/blastdb/nt_viruses.fasta.gz $(viral_reference_fastas_dir)`)

In [ ]:
# run(`ln -s $(homedir())/workspace/blastdb/ref_viruses_rep_genomes.fasta.gz $(viral_reference_fastas_dir)`)

In [ ]:
# output_file = joinpath(viral_reference_fastas_dir, "ictv-examplar-viruses.fna.gz")
# fasta_files = filter(x -> occursin(r"\.fna\.gz", x), readdir(joinpath(data_dir, "ictv_genomes"), join=true))
# Mycelia.merge_fasta_files(fasta_files=fasta_files, fasta_file=output_file)

In [ ]:
# human_reference = joinpath(data_dir, "GCF_000001405.40", "ncbi_dataset", "data", "GCF_000001405.40", "GCF_000001405.40_GRCh38.p14_genomic.fna.gz")
# run(`ln -s $(human_reference) $(viral_reference_fastas_dir)`)

In [ ]:
reference_fastas = filter(x -> occursin(r"\.f(na|asta)\.gz$", x), readdir(viral_reference_fastas_dir, join=true))

In [ ]:
sort!(reference_fastas, by=x->filesize(x))

In [ ]:
slurm_logdir = mkpath("$(homedir())/workspace/slurmlogs")

In [ ]:
# 4 & 5 too big to run on SCG3
for fasta_file in reference_fastas[1:3]
    Mycelia.sbatch(
        job_name = "bwa-mem2index.$(basename(fasta_file))",
        mail_user = "cameron.prybol@gmail.com",
        logdir = slurm_logdir,
        partition = "batch",
        account = "mpsnyder",
        mem_gb = 1536,
        cmd = "$(Mycelia.MAMBA) run --live-stream -n bwa-mem2 bwa-mem2 index $(fasta_file)")
end